BERT, ou Bidirectional Encoder Representations from Transformers, é um modelo de linguagem desenvolvido pelo Google que utiliza uma abordagem de aprendizado profundo para entender o contexto das palavras em uma frase. Ele é capaz de processar texto de forma bidirecional, o que melhora significativamente a precisão em tarefas de compreensão de linguagem natural.

As etapas dessa ponderada incluem: Carregdar e visualizar o conjunto de dado; tratamento e pré-processamento dos dados; tokenização ; construir o modelo utilizando o BERT pré treinado; e treinar e avaliar o modelo.

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive



In [21]:
!pip install transformers
!pip install tensorflow
!pip install tensorflow transformers pandas scikit-learn
!pip install gdown

#Carregamento dos dados e visualização inicial

In [22]:
# Conectar ao Google Drive

# Caminho do arquivo do Google Drive
file_path = '/content/drive/MyDrive/ponderada/bot_detection_data.csv'

# Carregar os dados
df = pd.read_csv(file_path)

In [23]:
df

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


#Exploração dos dados

Nesta etapa, realizei análises para compreender as principais informações do conjunto de dados, incluindo o tipo de dados em cada coluna, uma descrição estatística dos dados, a identificação e visualização dos valores ausentes por coluna, e a visualização dos valores únicos em cada coluna. O objetivo foi criar uma análise abrangente para entender detalhadamente o conteúdo do dataset e seu desenvolvimento.








In [24]:

print("\nInformações gerais sobre o dataset:")
print(df.info())



Informações gerais sobre o dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB
None


In [25]:

df.dtypes

,0
User ID,int64
Username,object
Tweet,object
Retweet Count,int64
Mention Count,int64
Follower Count,int64
Verified,bool
Bot Label,int64
Location,object
Created At,object


In [26]:
print("\n estatística do dataset:")
print(df.describe())



Descrição estatística do dataset:
             User ID  Retweet Count  Mention Count  Follower Count  \
count   50000.000000    50000.00000   50000.000000    50000.000000   
mean   548890.680540       50.00560       2.513760     4988.602380   
std    259756.681425       29.18116       1.708563     2878.742898   
min    100025.000000        0.00000       0.000000        0.000000   
25%    323524.250000       25.00000       1.000000     2487.750000   
50%    548147.000000       50.00000       3.000000     4991.500000   
75%    772983.000000       75.00000       4.000000     7471.000000   
max    999995.000000      100.00000       5.000000    10000.000000   

          Bot Label  
count  50000.000000  
mean       0.500360  
std        0.500005  
min        0.000000  
25%        0.000000  
50%        1.000000  
75%        1.000000  
max        1.000000  


In [27]:
print("\nValores ausentes por coluna:")
print(df.isnull().sum())


Valores ausentes por coluna:
User ID              0
Username             0
Tweet                0
Retweet Count        0
Mention Count        0
Follower Count       0
Verified             0
Bot Label            0
Location             0
Created At           0
Hashtags          8341
dtype: int64


In [28]:
print("\nValores únicos em cada coluna:")
print(df.nunique())


Valores únicos em cada coluna:
User ID           50000
Username          40566
Tweet             50000
Retweet Count       101
Mention Count         6
Follower Count     9923
Verified              2
Bot Label             2
Location          25199
Created At        49989
Hashtags          34247
dtype: int64


#Tratamento dos dados


Essa etepa será realizado para normalizar e limpar os tweets, eliminando caracteres especiais, links e outras informações irrelevantes. Além disso, procederemos com a remoção de colunas que não serão utilizadas no treinamento do modelo, com base na análise preliminar realizada. Abaixo estão as colunas selecionadas para exclusão








In [29]:
df = df.drop(columns=['User ID', 'Username', 'Location','Hashtags'])
df


,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Created At
0,Station activity person against natural majori...,85,1,2353,False,1,2020-05-11 15:29:50
1,Authority research natural life material staff...,55,5,9617,True,0,2022-11-26 05:18:10
2,Manage whose quickly especially foot none to g...,6,2,4363,True,0,2022-08-08 03:16:54
3,Just cover eight opportunity strong policy which.,54,5,2242,True,1,2021-08-14 22:27:05
4,Animal sign six data good or.,26,3,8438,False,1,2020-04-13 21:24:21
...,...,...,...,...,...,...,...
49995,Want but put card direction know miss former h...,64,0,9911,True,1,2023-04-20 11:06:26
49996,Provide whole maybe agree church respond most ...,18,5,9900,False,1,2022-10-18 03:57:35
49997,Bring different everyone international capital...,43,3,6313,True,1,2020-07-08 03:54:08
49998,Than about single generation itself seek sell ...,45,1,6343,False,0,2022-03-22 12:13:44


In [37]:
df = df.dropna(subset=['cleaned_tweet', 'Bot Label'])
df

,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Created At,cleaned_tweet
0,Station activity person against natural majori...,0.85,0.2,0.2353,False,1,2020-05-11 15:29:50,station activity person natural majority none ...
1,Authority research natural life material staff...,0.55,1.0,0.9617,True,0,2022-11-26 05:18:10,authority research natural life material staff...
2,Manage whose quickly especially foot none to g...,0.06,0.4,0.4363,True,0,2022-08-08 03:16:54,manage whose quickly especially foot none goal...
3,Just cover eight opportunity strong policy which.,0.54,1.0,0.2242,True,1,2021-08-14 22:27:05,cover eight opportunity strong policy
4,Animal sign six data good or.,0.26,0.6,0.8438,False,1,2020-04-13 21:24:21,animal sign six data good
...,...,...,...,...,...,...,...,...
49995,Want but put card direction know miss former h...,0.64,0.0,0.9911,True,1,2023-04-20 11:06:26,want put card direction know miss former half
49996,Provide whole maybe agree church respond most ...,0.18,1.0,0.9900,False,1,2022-10-18 03:57:35,provide whole maybe agree church respond base ...
49997,Bring different everyone international capital...,0.43,0.6,0.6313,True,1,2020-07-08 03:54:08,bring different everyone international capital...
49998,Than about single generation itself seek sell ...,0.45,0.2,0.6343,False,0,2022-03-22 12:13:44,single generation seek sell still hour


#Normalização

Normalizando os dados para garantir uma maior consistência e evitar viés em relação a alguma característica específica.








In [38]:
# Realizando a normalização das colunas numericas dos dados
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler  # Importar MinMaxScaler

# Baixar stopwords do NLTK, se ainda não foram baixadas
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

# Funções de limpeza
def remove_urls(tweet):
    return re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

def remove_mentions_and_hashtags(tweet):
    return re.sub(r'\@\w+|\#', '', tweet)

def preprocess_text(tweet):
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet.lower()

def clean_tweet(tweet):
    tweet = remove_urls(tweet)
    tweet = remove_mentions_and_hashtags(tweet)
    tweet = preprocess_text(tweet)
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    return tweet





In [40]:
scaler = MinMaxScaler()
numerical_cols = ['Retweet Count', 'Mention Count', 'Follower Count']

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df[numerical_cols].describe())

       Retweet Count  Mention Count  Follower Count
count   50000.000000   50000.000000    50000.000000
mean        0.500056       0.502752        0.498860
std         0.291812       0.341713        0.287874
min         0.000000       0.000000        0.000000
25%         0.250000       0.200000        0.248775
50%         0.500000       0.600000        0.499150
75%         0.750000       0.800000        0.747100
max         1.000000       1.000000        1.000000


#Tokenização



Processo para transformar os dados em formato adequado para o modelo BERT

In [42]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#Construção do modelo BERT

Na construção do modelo BERT, o primeiro passo foi dividir o conjunto de dados em conjuntos de treinamento e teste, utilizando 30% dos dados para teste. Para facilitar o treinamento, uma amostra de 5.000 tweets foi selecionada aleatoriamente tanto para o treinamento quanto para o teste. Em seguida, os dados foram tokenizados e codificados usando o BertTokenizer, com truncamento e preenchimento automáticos para se adequar ao modelo.

Os dados tokenizados foram convertidos para o formato TensorFlow, criando conjuntos de dados prontos para o treinamento, que foram embaralhados e divididos em lotes de 64. O modelo BERT pré-treinado foi inicializado para classificação de sequência com duas classes, e o modelo foi compilado com o otimizador Adam, uma função de perda de entropia cruzada esparsa e a métrica de acurácia.

Após compilar o modelo, ele foi treinado por duas épocas. Por fim, o modelo foi avaliado com os dados de teste, e as previsões foram obtidas. As métricas de precisão e acurácia foram calculadas e exibidas, fornecendo uma avaliação do desempenho do modelo na tarefa de classificação.








In [45]:
# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['Tweet'], df['Bot Label'], test_size=0.3, random_state=42
)

In [44]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score

# Inicializar o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Definindo amostragem para treinamento mais rápido
sample_size = 5000
X_train_sample = X_train.sample(n=sample_size, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

X_test_sample = X_test.sample(n=sample_size, random_state=42)
y_test_sample = y_test.loc[X_test_sample.index]

# Tokenizar e codificar os dados de treinamento e teste
train_encodings = tokenizer(list(X_train_sample), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(list(X_test_sample), truncation=True, padding=True, return_tensors='tf')

# Converter os dados de treinamento e teste para o formato TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train_sample)).shuffle(1000).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test_sample)).batch(64)

# Inicializar o modelo BERT pré-treinado
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

bert_model.fit(train_dataset, epochs=2)

y_pred_prob = bert_model.predict(test_dataset)
y_pred = np.argmax(y_pred_prob.logits, axis=1)

accuracy = accuracy_score(y_test_sample, y_pred)
precision = precision_score(y_test_sample, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
79/79 [==============================] - 1025s 12s/step - loss: 0.6967 - accuracy: 0.5090
Epoch 2/2
79/79 [==============================] - 326s 4s/step
Accuracy: 0.5012
Precision: 0.5012


**Análise inciial**: Apos realizar o treinamento do modelo BERT, os resultados obtidos para Accuracy (acurácia) e Precision (precis'ão) foram
0.5012 e 0.5012 respectivamente.



#Avaliação do modelo

Avaliação das métricas de desemoenho:

In [49]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# Avaliar o modelo
y_pred_prob = bert_model.predict(test_dataset)
y_pred_logits = y_pred_prob.logits  # Acesse os logits diretamente
y_pred = np.argmax(y_pred_logits, axis=1)

# Calcular as métricas
accuracy = accuracy_score(y_test_sample, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test_sample, y_pred, average='binary')
# Para AUC, use a probabilidade da classe positiva (1)
auc = roc_auc_score(y_test_sample, tf.nn.softmax(y_pred_logits, axis=-1).numpy()[:, 1])

# Exibir os resultados
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'AUC: {auc:.2f}')


79/79 [==============================] - 309s 4s/step
Accuracy: 0.50
Precision: 0.50
Recall: 1.00
F1 Score: 0.67
AUC: 0.50


A análise das métricas de desempenho revela os seguintes pontos:

* Acurácia (Accuracy: 0.50): A acurácia de 50% indica que o modelo está acertando metade das previsões, o que sugere que ele pode estar simplesmente classificando aleatoriamente, especialmente se as classes estiverem desbalanceadas.

* Precisão (Precision: 0.50): A precisão também é de 50%, significando que, das previsões feitas como positivas, apenas metade estava correta. Isso sugere que o modelo pode estar gerando muitos falsos positivos.

* Recall (Recall: 1.00): Um recall de 100% é um ponto forte, indicando que o modelo está identificando corretamente todos os casos positivos. No entanto, isso pode ser problemático se estiver custando a precisão, já que o modelo está classificando todos os exemplos como positivos.

* F1 Score (F1 Score: 0.67): O F1 score de 0.67 reflete um equilíbrio entre precisão e recall, mas ainda não é ideal. Isso sugere que, embora o modelo seja muito sensível (alto recall), ele não é muito preciso.

* AUC (AUC: 0.50): Um AUC de 0.50 é equivalente ao desempenho de uma escolha aleatória. Isso indica que o modelo não é eficaz em distinguir entre as classes.


Em resumo, o modelo apresenta um desempenho fraco, com alta sensibilidade, mas baixa precisão e acurácia. A presença de um recall perfeito sugere que o modelo pode estar se comportando de maneira enviesada, possivelmente classificando a maioria dos exemplos como positivos.

#Possíveis melhorias

Para aprimorar o desempenho do projeto, algumas estratégias podem ser consideradas:

* Balanceamento de Classes: Implementar técnicas de balanceamento, como oversampling da classe minoritária (por exemplo, SMOTE) ou undersampling da classe majoritária, pode ajudar a melhorar a precisão do modelo e reduzir a taxa de falsos positivos.

* Ajuste de Hiperparâmetros: Realizar uma busca mais aprofundada nos hiperparâmetros do modelo, como a taxa de aprendizado, o número de épocas e o tamanho do lote, pode otimizar o treinamento e levar a um melhor desempenho geral.

* Aprimoramento do Pré-processamento: Revisar e melhorar o pré-processamento dos dados, como a normalização e a limpeza de texto, pode aumentar a qualidade dos dados de entrada e, consequentemente, melhorar a performance do modelo.

* Modelos Alternativos: Considerar o uso de outros modelos de aprendizado de máquina ou técnicas de ensemble (como Random Forests ou Gradient Boosting) pode ser útil, especialmente se o BERT não estiver se ajustando bem ao conjunto de dados específico.